In [ ]:
import pandas as pd
import sys
import plotly.express as px

from datastory import DataStory

sys.path.append('../lib')

import utils, pesys_utils, pandas_utils

In [ ]:
utils.set_secrets_as_env()

In [ ]:
con.close()

In [ ]:
con = pesys_utils.open_pen_connection()
df = pandas_utils.pandas_from_sql(sqlfile='../sql/bpen002_oppg.sql', con=con, tuning=10000, lowercase=True)
con.close()

In [ ]:
df.head()

In [ ]:
px.bar(df, "dato", "antall", color="sakstype")

In [ ]:
df["dato"] = pd.to_datetime(df.dato)
df.set_index('dato',inplace=True)

In [ ]:
#sakstyper = df.sakstype.unique()
sakstyper = ['Alderspensjon', "Uføretrygd"]
figs = {}
for i in range(len(sakstyper)):
    ytelse = sakstyper[i]
    df_ytelse = df.query(f"sakstype=='{ytelse}'")
    df_mnd = df_ytelse.groupby("oppgavetekst")[["antall"]].resample('MS').sum().reset_index()
    figs[ytelse] = px.bar(df_mnd, "dato", "antall", color="oppgavetekst", title=ytelse)

In [ ]:
for _, fig in figs.items():
    fig.show()